Minimal example trying to get taxonomy information out of EMP communities

*Tom Smith 2022*

In [82]:
import biom
import pandas as pd

In [83]:
#loading biom table
table=biom.load_table('emp_deblur_150bp.subset_2k.rare_5000.biom')

In [84]:
#get all OTUs
obs_arr=(table.ids('observation'))
len(obs_arr)

91364

In [85]:
#get sample ids
sample_arr=table.ids('sample')
len(sample_arr) #number of samples in the table

975

Seems a bit suspicious, shouldn't there be 2000 sample IDs in the 2k subset? Oh well, we'll ignore that for now...

The biom table is like a table of OTUs X Sample, so we just need to know which OTUs appear in which sample?

In [86]:
sample_data=sample_arr[0] # subsetting to just the first sample ID
sample_freq=table.data(sample_data,axis='sample', dense=True) # "which elements of the OTU table have this sample id?"

In [87]:
# try to pull out the data associated with the OTUs within that sample
nz_ind=sample_freq !=0 #stores positions corresponding to OTUs where the sample frequency is non-zero
sample_otus=obs_arr[nz_ind] # get the OTUs 
taxa_present=[]
for otu in sample_otus[:]: # loop through each OTU within the sample and collect its taxonomy data
    taxa_present.append(table.metadata(otu,'observation')['taxonomy'])
len(taxa_present) # how many taxa are there?

1315

In [88]:
# put this into a dataframe
df_otu_result = pd.DataFrame.from_dict(taxa_present)
df_otu_result.columns = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
df_otu_result

,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,k__Bacteria,p__Acidobacteria,c__Acidobacteria-6,o__iii1-15,f__,g__,s__
1,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Luteimonas,s__
2,k__Bacteria,p__Actinobacteria,c__Thermoleophilia,o__Gaiellales,f__,g__,s__
3,k__Bacteria,p__Acidobacteria,c__Acidobacteria-6,o__iii1-15,f__mb2424,g__,s__
4,k__Bacteria,p__Actinobacteria,c__Thermoleophilia,o__Gaiellales,f__Gaiellaceae,g__,s__
...,...,...,...,...,...,...,...
1310,k__Bacteria,p__Proteobacteria,c__Deltaproteobacteria,o__Desulfuromonadales,f__Pelobacteraceae,g__,s__
1311,k__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhizobiales,f__Methylocystaceae,g__Methylosinus,s__
1312,k__Bacteria,p__Bacteroidetes,c__[Saprospirae],o__[Saprospirales],f__Chitinophagaceae,g__,s__
1313,k__Bacteria,p__Proteobacteria,c__Betaproteobacteria,None,None,None,None


In [89]:
# can write this out somewhere if we like:
df_otu_result.to_csv('results_otu.csv')

That was an example for just one of the communities in the dataset (samples). You'd want to do something like write a loop through all the sample IDs and pull out the OTU taxonomy from each sample.

Looks like a large proportion of these OTUs don't have taxonomy assigned to the species level though, this is likely to be a problem for our purposes....